In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

In [ ]:
import os
import cv2
import numpy as np
from glob import glob
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [ ]:
def load_data(path, split=0.1):
  images = sorted(glob(os.path.join(path, "images/*" )))
  masks = sorted(glob(os.path.join(path, "masks/*" )))

  total_size = len(images)
  valid_size = int(split * total_size)
  test_size = int(split * total_size)

  #print(total_size, valid_size, test_size)

  train_x, valid_x = train_test_split(images, test_size = valid_size, random_state = 42)
  train_y, valid_y = train_test_split(masks, test_size = valid_size, random_state = 42)

  train_x, test_x = train_test_split(train_x, test_size = test_size, random_state = 42)
  train_y, test_y = train_test_split(train_y, test_size = test_size, random_state = 42)

  return(train_x, train_y), (valid_x, valid_y), (test_x, test_y)

def read_image(path):
  path = path.decode()
  x = cv2.imread(path, cv2.IMREAD_COLOR)
  x = cv2.resize(x, (256,256))
  x = x/255.0    
  return x

def read_mask(path):
  path = path.decode()
  x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
  x = cv2.resize(x, (256,256))
  x = x/255.0
  x = np.expand_dims(x, axis = -1)    
  return x

def tf_parse(x, y):
  def _parse(x, y):
    x = read_image(x)
    y = read_mask(y)
    return x, y

  x, y = tf.numpy_function(_parse, [x, y], [tf.float64, tf.float64])
  x.set_shape([256, 256, 3])
  y.set_shape([256, 256, 1])

  return x, y

def tf_dataset(x, y, batch=8):
  dataset = tf.data.Dataset.from_tensor_slices((x, y))
  dataset = dataset.map(tf_parse)
  dataset = dataset.batch(batch)
  dataset = dataset.repeat()
  return dataset

In [ ]:
path = "/content/drive/MyDrive/Polyp Work/Dataset"
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(path)
print(len(train_x), len(valid_x), len(test_x))

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, concatenate, BatchNormalization, Activation, Conv2DTranspose, Add
from tensorflow.keras.models import Model

def conv_block(inputs, n_filters, kernel_size=3, strides=1):
    x = Conv2D(n_filters, kernel_size, strides=strides, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def residual_block(inputs, n_filters):
    x = conv_block(inputs, n_filters)
    x = conv_block(x, n_filters)
    x = Add()([x, inputs])
    return x

def upconv_block(inputs, n_filters, kernel_size=3, strides=2):
    x = Conv2DTranspose(n_filters, kernel_size, strides=strides, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def ResUnetPlusPlus():
    input_shape=(256, 256, 3)
    inputs = Input(input_shape)
    n_classes=1
    start_filters=32 
    depth=4

    skip_connections = []
    x = inputs

    # Encoder
    for i in range(depth):
        x = conv_block(x, start_filters * 2 ** i, kernel_size=3, strides=1)
        x = residual_block(x, start_filters * 2 ** i)
        skip_connections.append(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)

    x = conv_block(x, start_filters * 2 ** depth, kernel_size=3, strides=1)

    # Decoder
    for i in reversed(range(depth)):
        x = upconv_block(x, start_filters * 2 ** i, kernel_size=3, strides=2)
        x = concatenate([x, skip_connections[i]])
        x = conv_block(x, start_filters * 2 ** i, kernel_size=3, strides=1)
        x = residual_block(x, start_filters * 2 ** i)

    # Output
    outputs = Conv2D(n_classes, kernel_size=1, strides=1, activation='sigmoid')(x)

    model = Model(inputs, outputs)
    return model

In [ ]:
model = ResUnetPlusPlus()
model.summary()

In [ ]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard
from tensorflow.keras.metrics import Recall, Precision

In [ ]:
def iou(y_true, y_pred):
  def f(y_true, y_pred):
    intersection = (y_true * y_pred).sum()
    union = y_true.sum() + y_pred.sum() - intersection
    x = (intersection + 1e-15) / (union + 1e-15)
    x = x.astype(np.float32)
    return x
  return tf.numpy_function(f, [y_true, y_pred], tf.float32)

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)
batch = 8
lr = 1e-4
epochs = 100

train_dataset = tf_dataset(train_x, train_y, batch = batch)
valid_dataset = tf_dataset(valid_x, valid_y, batch = batch)

opt = tf.keras.optimizers.Adam(lr)
metrics = ["acc", Recall(), Precision(), iou]
model.compile(loss = "binary_crossentropy", optimizer = opt, metrics = metrics)

callbacks = [ ModelCheckpoint("/content/drive/MyDrive/Polyp Work/2_ResUNet++.hdf5"),
    ReduceLROnPlateau(monitor = "val_loss", factor = 0.1, patience = 3),
    TensorBoard(),
    EarlyStopping(monitor = "val_loss", patience = 5, restore_best_weights = False)
]


train_steps = len(train_x) // batch
valid_steps = len(valid_x) // batch

if len(train_x) % batch !=0:
    train_steps += 1

if len(valid_x) % batch !=0:
    valid_steps += 1
  
history = model.fit(
    train_dataset, 
    validation_data = valid_dataset,
    epochs = epochs,
    steps_per_epoch = train_steps,
    validation_steps = valid_steps,
    callbacks = callbacks,
    shuffle = False
  )
  

In [ ]:
model.save("/content/drive/MyDrive/Polyp Work/2_ResUNet++.hdf5")

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
acc = history.history['acc'][-1]
loss = history.history['loss'][-1]
print('Training Accuracy:',acc);
print('Training loss:',loss)

Training Accuracy: 0.9927931427955627
Training loss: 0.008807224221527576


In [ ]:
from tensorflow.keras.utils import CustomObjectScope

test_dataset = tf_dataset(test_x, test_y, batch = batch)
test_steps = len(test_x)//batch
if len(test_x) % batch != 0:
    test_steps += 1

with CustomObjectScope({'iou': iou}):
  model = tf.keras.models.load_model("/content/drive/MyDrive/Polyp Work/2_ResUNet++.hdf5")

model.evaluate(test_dataset, steps = test_steps)

In [ ]:
def read_image(path):
  x = cv2.imread(path, cv2.IMREAD_COLOR)
  y = cv2.cvtColor(x, cv2.COLOR_RGB2BGR)
  y = cv2.resize(y, (256,256))
  y = y/255.0
  x = cv2.resize(x, (256,256))
  x = x/255.0    #(256, 256, 3)
  return x, y

def read_mask(path):
  x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
  x = cv2.resize(x, (256,256))
  x = np.expand_dims(x, axis = -1)    #(256, 256, 1)
  return x

def mask_parse(mask):
    mask = np.squeeze(mask)
    mask = [mask, mask, mask]
    mask = np.transpose(mask, (1, 2, 0))
    return mask

In [ ]:
import cv2
import matplotlib.pyplot as plt

In [ ]:
from tqdm import tqdm
for i, (x, y) in tqdm(enumerate(zip(test_x, test_y)), total = len(test_x)):
        x, rgb_img = read_image(x)
        y = read_mask(y)
                
        y_pred = model.predict(np.expand_dims(x, axis = 0))
        y_pred = y_pred[0] > 0.5
        h, w, _ = x.shape
        
        #x = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        white_line = np.ones((h, 10, 3)) * 255.0
        
        all_images = [
            rgb_img, white_line, 
            mask_parse(y), white_line,
            mask_parse(y_pred) * 255.0
        ]
        
        image = np.concatenate(all_images, axis = 1)
        
        plt.imshow(image)
        plt.show()
        cv2.imwrite(f"/content/drive/MyDrive/Polyp_Work/Result_ResUNet++/{i}.png", image)

In [ ]:
pip install keract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
img_path = '/content/drive/MyDrive/Polyp_Work/CVC-ClinicDB/images/1.png'

In [ ]:
from PIL import Image

def preprocess_image(img_path, target_size):
    img = Image.open(img_path)
    img = img.resize(target_size)
    x = np.array(img)
    return x

In [ ]:
from keract import get_activations
x = np.expand_dims(x, axis=0)
#x = np.squeeze(x, axis=0)
activations = get_activations(model, x)

In [ ]:
from keract import display_activations
display_activations(activations, save=False)

In [ ]:
import keras
import matplotlib.pyplot as plt
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import array_to_img
import numpy

with CustomObjectScope({'iou': iou}):
  model = tf.keras.models.load_model("/content/drive/MyDrive/Polyp_Work/Res_UNet++.hdf5")

#img = plt.imread('/content/drive/MyDrive/Polyp_Work/CVC-ClinicDB/images/1.png')

In [ ]:
last_layer = model.get_layer('conv2d_25').output
heatmap_model = keras.models.Model(inputs=model.input, outputs=last_layer)

In [ ]:
import cv2
import numpy as np

img = cv2.imread('/content/drive/MyDrive/Polyp_Work/CVC-ClinicDB/images/488.png')
img = cv2.resize(img, (256, 256))
img = np.expand_dims(img, axis=0)

heatmap_activations = heatmap_model.predict(img)

heatmap = np.mean(heatmap_activations, axis=-1)


In [ ]:
heatmap = cv2.resize(heatmap, (256, 256))
heatmap = cv2.normalize(heatmap, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)
if heatmap.ndim > 2:
    heatmap = cv2.split(heatmap)[0]  # select one channel
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
#heatmap = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_JET)

result = cv2.addWeighted(cv2.cvtColor(np.uint8(255*img[0]), cv2.COLOR_RGB2BGR), 0.5, heatmap, 0.5, 0)

In [ ]:
from google.colab.patches import cv2_imshow
cv2_imshow(result)
cv2.waitKey(0)